### El  DataSet abalone_alumno.csv corresponde a datos sobre las “orejas de mar”, gasterópodos marinos del género Haliotis. Es posible calcular su edad contando los anillos o Rings, de tal forma que cada ring equivale a 1,5 años de edad. Sin embargo el proceso de contar estos anillos es complicado, nuestra intención es ver si podemos pronosticar la cantidad de anillos en base a otras features que son fáciles de medir, como las provistas en el dataset. 

Vamos a armar un Ensemble con los siguientes modelos:
•	Regresión Logística con Ridge
•	SVM
•	kNN
•	RandomForest 

Por cuestiones de tiempo utilice un solo hiperparámetro por modelo.
Mediante el uso de GridSearchCV y un esquema de votación adecuado deberá calcular qué RMSE obtenemos con este ensemble para nuestro problema.


### Aplicando regresion...

In [157]:
df=pd.read_csv('data/abalone_alumno.csv')
df.head()

,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Rings,Sex_F,Sex_I,Sex_M
0,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15.0,0,0,1
1,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7.0,0,0,1
2,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9.0,1,0,0
3,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10.0,0,0,1
4,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7.0,0,1,0


Separo train y tests

In [158]:
X=df.drop(axis=1,columns='Rings')
y=df['Rings']

In [159]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=20,random_state=123)

Estandarizo para no tener tanto efecto de outliers

In [160]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

### Aplicando regresion...

In [161]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import VotingRegressor

Separo train y tests

In [162]:
X=df.drop(axis=1,columns='Rings')
y=df['Rings']

In [163]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=20,random_state=123)

Creo los modelos

In [164]:
ridge=Ridge(max_iter= 20000,fit_intercept=True, random_state=123)
knn=KNeighborsRegressor(n_jobs=-1)
rf=RandomForestRegressor(n_jobs=-1,random_state=123)
svm=SVR()

Creacion del modelo ensamblado

In [165]:
modelos=[('ridge',ridge),('knn',knn),('rf',rf),('svm',svm)]

ensemble=VotingRegressor(estimators=modelos,n_jobs=-1) #, weights=[2,1,3,1])

Creo el diccionario con los parametros que quiero pasarle al meta modelo

In [166]:
hiperparametros={}
hiperparametros['ridge__alpha']=[1e-3,1,1e3] # [1e-4,1e-3,1e-2,1e-1,1,1e1,1e2,1e3]
hiperparametros['knn__n_neighbors']=[10,20,30]  #[1,2,3,4,5,6,7,8,9,10,11,12,13]
hiperparametros['rf__n_estimators']=[100,200,300]  # [1,2,3,4,5,6,7,8,9,10,11,12,13]
hiperparametros['svm__C']=[1e-3,1,1e3] # [1e-4,1e-3,1e-2,1e-1,1,1e1,1e2,1e3]
hiperparametros

{'ridge__alpha': [0.001, 1, 1000.0],
 'knn__n_neighbors': [10, 20, 30],
 'rf__n_estimators': [100, 200, 300],
 'svm__C': [0.001, 1, 1000.0]}

Le paso el meta modelo y los hiperparametros a grid search CV

In [167]:
from sklearn.model_selection import GridSearchCV

grid_ensemble= GridSearchCV(estimator=ensemble, param_grid=hiperparametros, cv=5,n_jobs=-1, return_train_score=True)

Entrenamos nuestro metamodelo con los hiperparametros seleccionados y haciendo CV

In [168]:
grid_ensemble.fit(X_train, y_train);

Mejores resultados:

In [169]:
grid_ensemble.best_score_

0.5808126970438657

Mejores parametros obtenidos:

In [170]:
grid_ensemble.best_params_

{'knn__n_neighbors': 20,
 'rf__n_estimators': 300,
 'ridge__alpha': 0.001,
 'svm__C': 1000.0}

Evaluamos en el test:

In [171]:
y_pred = grid_ensemble.predict(X_test)
RMSE = np.sqrt(mean_squared_error(y_test,y_pred))
print(RMSE)

1.9418231820623555


Guardo en pickle

In [172]:
import pickle

nombre_archivo='modelo_mejor.sav'

pickle.dump(grid_ensemble, open(nombre_archivo, 'wb'))

#### Conclusion: el RMSE mejoro al que obtuve en los metodos individuales!